<a href="https://colab.research.google.com/github/pomoq-dev/video-clipping/blob/master/notebooks/mozilla_deepshpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

DIR_AUDIOS_VIDEOS = '/content/drive/MyDrive/INSTALL_VIDEOS'
drive.mount('/content/drive')

In [ ]:
print('Git clone project and install requirements...')
!git clone https://github.com/pomoq-dev/video-clipping.git &> /dev/null
%cd video-clipping/

!export PYTHONPATH=/content/video-clipping:$PYTHONPATH
!wget https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -O /usr/local/bin/yt-dlp
!chmod a+rx /usr/local/bin/yt-dlp

!pip install -r requirements.txt

In [ ]:
!pip install deepspeech
!deepspeech --version

In [ ]:
# TODO 0.9.3
!mkdir deepspeech-0.8.2-models
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.8.1/deepspeech-0.8.1-models.pbmm
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.8.1/deepspeech-0.8.1-models.scorer
!mv deepspeech-0.8.1-models.pbmm deepspeech-0.8.1-models.scorer deepspeech-0.8.2-models/
!ls -l deepspeech-0.8.2-models/

In [ ]:
import wave
import deepspeech
import numpy as np
from vidlib import audio_tools

model_file_path = 'deepspeech-0.8.2-models/deepspeech-0.8.1-models.pbmm'
model = deepspeech.Model(model_file_path)
scorer_file_path = 'deepspeech-0.8.2-models/deepspeech-0.8.1-models.scorer'
model.enableExternalScorer(scorer_file_path)

lm_alpha = 0.75
lm_beta = 1.85
model.setScorerAlphaBeta(lm_alpha, lm_beta)

beam_width = 500
model.setBeamWidth(beam_width)

filename = 'input.wav'
audio_tools.m4a2wav(os.path.join(DIR_AUDIOS_VIDEOS, 'audio0.m4a'), filename)
w = wave.open(filename, 'r')
rate = w.getframerate()
frames = w.getnframes()
buffer = w.readframes(frames)

print(rate)
print(model.sampleRate())
print(str(type(buffer)))

data16 = np.frombuffer(buffer, dtype=np.int16)
print(str(type(data16)))

text = model.stt(data16)
print(text)